In [2]:
import pandas as pd
import torch
from tqdm.auto import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
import numpy as np

In [3]:
path_to_data = "/Users/abdoulabdillahi/Desktop/Thesis/Bio_project/200_samples_with_encoded.csv"
df = pd.read_csv(path_to_data)
gene_cols = [c for c in df.columns if c.startswith("gene_")]

In [7]:
df.head()

,Run accession,Ciprofloxacin_NS,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,...,gene_15619,gene_15620,gene_15621,gene_15622,gene_15623,gene_15624,gene_15625,gene_15626,gene_15627,gene_15628
0,ERR4035837,1,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],[nan nan nan nan nan nan nan nan nan nan nan n...,...,['-' '-' 'g' 'a' 't' 'g' 'a' 'a' 'a' 'a' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],['a' 't' 'g' ... 't' 'g' 'a'],['-' '-' '-' '-' '-' '-' 'c' 't' 'g' 't' 'a' '...,['a' 't' 'a' 't' 'c' 'c' 'c' 'a' 'g' 'a' 't' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...
1,ERR4034242,1,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan ... nan nan nan],[nan nan nan nan nan nan nan nan nan nan nan n...,...,['-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],['a' 't' 'g' ... 't' 'g' 'a'],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 't' 'c' 'c' 'c' 'a' 'g' 'a' 't' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...
2,ERR4034731,1,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan ... nan nan nan],[nan nan nan nan nan nan nan nan nan nan nan n...,...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan ... nan nan nan],[nan nan nan ... nan nan nan],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 't' 'c' 'c' 'c' 'a' 'g' 'a' 't' '...,['a' 't' 'g' 'c' 'g' 'g' 'c' 't' 't' 'g' 'c' '...,['a' 't' 'g' 'c' 'a' 'g' 't' 't' 't' 'g' 't' '...,['a' 't' 'g' 't' 'c' 'g' 'g' 'c' 'c' 'a' 'c' '...
3,ERR4036152,0,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],[nan nan nan nan nan nan nan nan nan nan nan n...,...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' ... 't' 'g' 'a'],['a' 't' 'g' ... 't' 'g' 'a'],[nan nan nan nan nan nan nan nan nan nan nan n...,['a' 't' 'g' 't' 'c' 'c' 'c' 'a' 'g' 'a' 't' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...
4,ERR4036469,0,['a' 't' 'g' 't' 'c' 't' 'a' 't' 'a' 'c' 'a' '...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan nan nan nan nan nan nan nan nan n...,[nan nan nan ... nan nan nan],[nan nan nan nan 

In [8]:
def join_genes(row):
    toks = []
    for g in gene_cols:
        cell = row[g]
        if pd.isna(cell): continue
        if isinstance(cell, str) and cell.startswith("["):
            seq = [t for t in cell.strip("[]").split() if t.lower()!="nan"]
        else:
            seq = list(cell)
        toks.append("".join(seq))
    return "".join(toks)

df["text"]  = df.apply(join_genes, axis=1)
df["label"] = df["Ciprofloxacin_NS"].astype(int)

In [18]:
df['text'][:1]

0    'a''t''g''t''c''t''a''t''a''c''a''g''a''a''c''...
Name: text, dtype: object

In [21]:
df['label'].unique()

array([1, 0])

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        # squeeze batch dimension:
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [ ]:
checkpoint = "zhihan1996/DNABERT-2-117M"
tokenizer  = AutoTokenizer.from_pretrained(checkpoint, use_fast=False, trust_remote_code=True)
model      = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2, trust_remote_code=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.embeddings.position_embeddings.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.1.attention.self.key.bias', 'bert.encoder.layer.1.attention.self.key.weight', 'bert.encoder.layer.1.attention.self.query.bias', 'bert.encoder.layer.1.attention.self.query.weight', 'bert.encoder.layer.1.at

In [ ]:
dataset = SequenceDataset(df["text"].tolist(),
                          df["label"].tolist(),
                          tokenizer,
                          max_length=512)

train_size = int(0.8 * len(dataset))
val_size   = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(
    train_ds, batch_size=8, shuffle=True,
    collate_fn=lambda batch: tokenizer.pad(batch, return_tensors="pt")
)
val_loader = DataLoader(
    val_ds, batch_size=16, shuffle=False,
    collate_fn=lambda batch: tokenizer.pad(batch, return_tensors="pt")
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 3  # num_epochs=3
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps//10,
    num_training_steps=total_steps
)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_epoch():
    model.train()
    total_loss, total_correct, total_count = 0.0, 0, 0
    loop = tqdm(train_loader, desc="Training", leave=False)
    for batch in loop:
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # metrics
        preds = logits.argmax(dim=1)
        correct = (preds == batch["labels"]).sum().item()
        bs = batch["labels"].size(0)

        total_loss    += loss.item() * bs
        total_correct += correct
        total_count   += bs

        loop.set_postfix({
            "loss": f"{loss.item():.4f}",
            "batch_acc": f"{correct/bs:.2%}"
        })

    avg_loss = total_loss / total_count
    avg_acc  = total_correct / total_count
    return avg_loss, avg_acc

def eval_epoch():
    model.eval()
    total_loss, total_correct, total_count = 0.0, 0, 0
    loop = tqdm(val_loader, desc="Evaluating", leave=False)
    with torch.no_grad():
        for batch in loop:
            batch = {k:v.to(device) for k,v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            preds = logits.argmax(dim=1)
            correct = (preds == batch["labels"]).sum().item()
            bs = batch["labels"].size(0)

            total_loss    += loss.item() * bs
            total_correct += correct
            total_count   += bs

            loop.set_postfix({
                "loss": f"{loss.item():.4f}",
                "batch_acc": f"{correct/bs:.2%}"
            })

    avg_loss = total_loss / total_count
    avg_acc  = total_correct / total_count
    return avg_loss, avg_acc

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch()
    val_loss,   val_acc   = eval_epoch()
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Train — loss: {train_loss:.4f}, acc: {train_acc:.4%}")
    print(f"  Val   — loss: {val_loss:.4f}, acc: {val_acc:.4%}")


KeyboardInterrupt: 